In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler

In [ ]:
customer = pd.read_csv('marketing_customer_analysis.csv')

# Lab | Customer Analysis Round 4

##### 1. Check the data types of the columns. Get the numeric data into dataframe called numerical and categorical columns in a dataframe called categoricals. (You can use np.number and np.object to select the numerical data types and categorical data types respectively)

In [ ]:
customer.dtypes

In [ ]:
cus_num = customer.select_dtypes(include = np.number)


In [ ]:
cus_cat = customer.select_dtypes(include = np.object)


##### 2. Now we will try to check the normality of the numerical variables visually
##### 2. 1 Use seaborn library to construct distribution plots for the numerical variables

In [ ]:
# Check 1

for col in cus_num:
    sns.displot(cus_num[col])

##### 2.2 Use Matplotlib to construct histograms

In [ ]:
for col in cus_num:
    
    plt.figure(figsize=(8, 5))  
    
    plt.hist(cus_num[col], bins=20)
    
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

##### 2.3 Do the distributions for different numerical variables look like a normal distribution

In [ ]:
cus_num

In [ ]:
def log_transfom_clean(x):
    x = np.log10(x)
    if np.isfinite(x):
        return x
    else:
        return 0

In [ ]:
for col in cus_num:
    plt.figure(figsize=(7, 4))
    plt.hist(cus_num[col].apply(log_transfom_clean), bins=30)   
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

In [ ]:
def log_transform_clean2(x):
    x = np.log10(x)
    if np.isfinite(x):
        return x
    else:
        return np.NAN

In [ ]:
for col in cus_num:
    
    cus_num[col] = list(map(log_transform_clean2, cus_num[col]))
    cus_num[col] = cus_num[col].fillna(np.mean(cus_num[col]))
    
    plt.figure(figsize=(7, 4))
    plt.hist(cus_num[col].apply(log_transfom_clean), bins=30)   
    plt.title(f'Histogram of {col}')
    plt.xlabel(col)
    plt.ylabel('Frequency')
    plt.show()

cus_num

I don´t think they could look as normal distributions using the log function. I could cut the values of the total claim amount at 600 but I wouldn´t do that because it provides lot of information. Here is the demonstration:

In [ ]:
sns.boxplot(cus_num['Total Claim Amount'])
plt.show()

In [ ]:
iqr = np.percentile(cus_num['Total Claim Amount'],75) - np.percentile(cus_num['Total Claim Amount'],25)

upper_limit = np.percentile(cus_num['Total Claim Amount'],75) + 1.5*iqr
lower_limit = np.percentile(cus_num['Total Claim Amount'],25) - 1.5*iqr

print(upper_limit)
print(lower_limit)

In [ ]:
cus_claim = cus_num[(cus_num['Total Claim Amount']>lower_limit)&(cus_num['Total Claim Amount']<2500)]
sns.displot(cus_claim['Total Claim Amount'])
plt.show()

In [ ]:
cus_claim['Total Claim Amount'] = list(map(log_transform_clean2, cus_claim['Total Claim Amount']))
cus_claim['Total Claim Amount'] = cus_claim['Total Claim Amount'].fillna(np.mean(cus_claim['Total Claim Amount']))
sns.displot(cus_claim['Total Claim Amount'])
plt.show()

##### 3. For the numerical variables, check the multicollinearity between the features. Please note that we will use the column total_claim_amount later as the target variable.


In [ ]:
# Reset data frame

cus_num = customer.select_dtypes(include = np.number)

In [ ]:
# Multicollinearity is a statistical phenomenon that occurs when two or more independent variables in a regression model are highly correlated with each other.

# We can create a matrix
correlation_matrix = cus_num.corr()
correlation_matrix


In [ ]:
# Or a heatmap

plt.figure(figsize=(8, 6))

sns.heatmap(correlation_matrix, annot=True)
plt.title('Correlation Matrix Heatmap')
plt.show()

The values are not very correlated in general.

Looking at the Total Claim Amount, we can say that is quite correlated to the Monthly Premium Auto. The Customer Lifetime Value provides some information with a coefficient of correlation of 0.23.

Also, the income coefficient tells us about a negative correlation, which means that the Total Claim Amount gets higher when the Income amount goes lower.

##### 4. Drop one of the two features that show a high correlation between them (greater than 0.9). Write code for both the correlation matrix and for seaborn heatmap. If there is no pair of features that have a high correlation, then do not drop any features

I´m going to drop the values with very little correlation.

In [ ]:
cus_corr = cus_num.drop(['Months Since Last Claim', 'Months Since Policy Inception', 'Number of Open Complaints', 'Number of Policies'], axis=1)
cus_corr

In [ ]:
correlation_matrix2 = cus_corr.corr()
correlation_matrix2

In [ ]:
plt.figure(figsize=(4, 3))
sns.heatmap(correlation_matrix2, annot=True)
plt.title('Correlation Matrix Heatmap 2')
plt.show()

# Lab | Customer Analysis Round 5

### Processing Data

##### 1. Further processing

I will compare the target value against the most correlated one, the Monthly Premium Auto.

In [ ]:
cus_num = customer.select_dtypes(include = np.number)

y = cus_num['Total Claim Amount']
X = cus_num[['Monthly Premium Auto']]

lm = linear_model.LinearRegression()
lm.fit(X,y)

regression_line = lm.intercept_ + lm.coef_[0]*cus_num['Monthly Premium Auto']

plt.plot(cus_num['Monthly Premium Auto'], regression_line, c = 'orange')
sns.scatterplot(x='Monthly Premium Auto',y='Total Claim Amount',data=cus_num)
plt.show()

I´ll check the R2-score

In [ ]:
print("R2-score is ", lm.score(X,y))
print("mean squared error (MSE) is ", mean_squared_error(lm.predict(X),y))

The R2-socre indicates the model does not explain very well the variance in the target variable.
Also, we can see by the MSE that the model's predictions are not very close to the true values.

##### 2. X-y split.

Already done above

##### 3. Normalize (numerical).

In [ ]:
y = cus_num['Total Claim Amount']
X = cus_num.drop(['Total Claim Amount'], axis=1)

X_num = X.select_dtypes(include = np.number)
X_cat = X.select_dtypes(include = object)

transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
print(x_normalized.shape)
x_normalized

cus_nor = pd.DataFrame(x_normalized, columns=X_num.columns)

# Lab | Customer Analysis Round 6

### Befor starting - Cells I have to run from previous labs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler

In [2]:
customer = pd.read_csv('marketing_customer_analysis.csv')
cus_num = customer.select_dtypes(include = np.number)
cus_cat = customer.select_dtypes(include = np.object)
y = cus_num['Total Claim Amount']
X = cus_num.drop(['Total Claim Amount'], axis=1)
X_num = X.select_dtypes(include = np.number)
transformer = MinMaxScaler().fit(X_num)
x_normalized = transformer.transform(X_num)
cus_nor = pd.DataFrame(x_normalized, columns=X_num.columns)
cus_nor.describe()

C:\Users\PC\AppData\Local\Temp\ipykernel_11860\3843852013.py:3: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cus_cat = customer.select_dtypes(include = np.object)


,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies
count,9134.000000,9134.000000,9134.000000,9134.000000,9134.000000,9134.000000,9134.000000
mean,0.074999,0.376645,0.135946,0.431343,0.485501,0.076878,0.245771
std,0.084382,0.303857,0.145181,0.287807,0.281879,0.182077,0.298773
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.025744,0.000000,0.029536,0.171429,0.242424,0.000000,0.000000
50%,0.047677,0.338959,0.092827,0.400000,0.484848,0.000000,0.125000
75%,0.086754,0.623318,0.202532,0.657143,0.717172,0.000000,0.375000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Processing Data - Deciding the Method

In [3]:
cus_cat.head()

,Customer,State,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Location Code,Marital Status,Policy Type,Policy,Renew Offer Type,Sales Channel,Vehicle Class,Vehicle Size
0,BU79786,Washington,No,Basic,Bachelor,2/24/11,Employed,F,Suburban,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car,Medsize
1,QZ44356,Arizona,No,Extended,Bachelor,1/31/11,Unemployed,F,Suburban,Single,Personal Auto,Personal L3,Offer3,Agent,Four-Door Car,Medsize
2,AI49188,Nevada,No,Premium,Bachelor,2/19/11,Employed,F,Suburban,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car,Medsize
3,WW63253,California,No,Basic,Bachelor,1/20/11,Unemployed,M,Suburban,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV,Medsize
4,HB64268,Washington,No,Basic,Bachelor,2/3/11,Employed,M,Rural,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car,Medsize


In [4]:
for col in cus_cat:
    print(cus_cat[col].value_counts())

BU79786    1
PU81096    1
CO75086    1
WW52683    1
XO38850    1
          ..
HS14476    1
YL91587    1
CT18212    1
EW35231    1
Y167826    1
Name: Customer, Length: 9134, dtype: int64
California    3150
Oregon        2601
Arizona       1703
Nevada         882
Washington     798
Name: State, dtype: int64
No     7826
Yes    1308
Name: Response, dtype: int64
Basic       5568
Extended    2742
Premium      824
Name: Coverage, dtype: int64
Bachelor                2748
College                 2681
High School or Below    2622
Master                   741
Doctor                   342
Name: Education, dtype: int64
1/10/11    195
1/27/11    194
2/14/11    186
1/26/11    181
1/17/11    180
1/19/11    179
1/31/11    178
1/3/11     178
1/20/11    173
2/26/11    169
1/28/11    169
2/19/11    168
1/5/11     167
2/27/11    167
1/11/11    166
2/4/11     164
2/10/11    161
2/28/11    161
1/2/11     160
1/21/11    160
1/29/11    160
2/22/11    158
2/5/11     158
2/3/11     158
2/7/11     157
2/12/11   

### Processing Data - OneHot Encoding

In [5]:
# cus_one = cus_cat.drop(['Customer', 'State', 'Effective To Date', 'EmploymentStatus', 'Policy', 'Renew Offer Type', 'Sales Channel', 'Vehicle Class'], axis=1)
cus_one = cus_cat.drop(['Customer', 'Effective To Date'], axis=1)
encoder = OneHotEncoder(drop='first').fit(cus_one)
encoded = encoder.transform(cus_one).toarray()

# for col in cus_one:
    # onehot_encoded = pd.DataFrame(encoded,columns=[cus_one[col].unique()])

onehot_encoded = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())
# onehot_encoded = pd.DataFrame(encoded,columns=['Female', 'Male','U'])
onehot_encoded.head()



,State_California,State_Nevada,State_Oregon,State_Washington,Response_Yes,Coverage_Extended,Coverage_Premium,Education_College,Education_Doctor,Education_High School or Below,...,Sales Channel_Branch,Sales Channel_Call Center,Sales Channel_Web,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Medsize,Vehicle Size_Small
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


### Processing Data - Columns excluded in the model

I´m going to exclude the Customer identification and the effective dat as I don´t consider them valuable information for the model.

### Processing Data - Concatenate

In [6]:
customer2 = pd.concat([cus_nor, onehot_encoded], axis=1)
customer2

,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,State_California,State_Nevada,State_Oregon,...,Sales Channel_Branch,Sales Channel_Call Center,Sales Channel_Web,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Medsize,Vehicle Size_Small
0,0.010629,0.562847,0.033755,0.914286,0.050505,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.062406,0.000000,0.139241,0.371429,0.424242,0.0,0.875,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.134960,0.487763,0.198312,0.514286,0.383838,0.0,0.125,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.070589,0.000000,0.189873,0.514286,0.656566,0.0,0.750,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.011245,0.438443,0.050633,0.342857,0.444444,0.0,0.000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9129,0.264137,0.719547,0.050633,0.514286,0.898990,0.0,0.125,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9130,0.014719,0.216081,0.075949,0.400000,0.282828,0.0,0.000,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9131,0.076951,0.000000,0.101266,0.257143,0.373737,0.6,0.125,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
9132,0.069098,0.219452,0.147679,0.971429,0.030303,0.0,0.250,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Linear Regression - Train-test split

In [7]:
y = customer['Total Claim Amount']
X = customer2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

### Linear Regression - Apply linear regression

In [8]:
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

0.7715935860091137

### Model Validation - R2

In [9]:
predictions_test = lm.predict(X_test)
r2_score(y_test, predictions_test)

0.7783428743713389

### Model Validation - MSE


In [10]:
predictions_test = lm.predict(X_test)
mse=mean_squared_error(y_test,predictions_test)
mse

20353.603333676998

### Model Validation - RMSE

In [11]:
rmse = np.sqrt(mean_squared_error(y_test,predictions_test))
rmse

142.66605529584464

### Model Validation - MAE

In [12]:
mae = mean_absolute_error(y_test,predictions_test)
mae

98.9051747662496

# Lab | Customer Analysis Round 7

### Reset variables

In [ ]:
customer = pd.read_csv('marketing_customer_analysis.csv')
cus_num = customer.select_dtypes(include = np.number)
cus_cat = customer.select_dtypes(include = np.object)
y = cus_num['Total Claim Amount']
X = cus_num.drop(['Total Claim Amount'], axis=1)
# X_num = X.select_dtypes(include = np.number)
# X_cat = X.select_dtypes(include = object)
cus_one = cus_cat.drop(['Customer', 'Effective To Date'], axis=1)
encoder = OneHotEncoder(drop='first').fit(cus_one)
encoded = encoder.transform(cus_one).toarray()
onehot_encoded = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())

### Model Normalised with StandardScaler - All columns

In [ ]:
Standardtransformer = StandardScaler().fit(cus_num)
X_standardized = Standardtransformer.transform(cus_num)

cus_std = pd.DataFrame(X_standardized,columns=cus_num.columns)
cus_std

In [ ]:
customer3 = pd.concat([cus_std, onehot_encoded], axis=1)
customer3

In [ ]:
y = customer['Total Claim Amount']
X = customer3

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001)

lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

predictions = lm.predict(X_train)
r2_score(y_train, predictions)

In [ ]:
predictions_test = lm.predict(X_test)
r2_score(y_test, predictions_test)

The model has a perfect R2 score. I´ve been checking if I had commited a mistake. I´ve done the standardscaler twice in a dfferent notebook, with all reset and I´ve also checked this:

- Overfitting: I didn´t drop outliers.

- I tested the model with test size=0.001 and it gives the same result

I have checked that this could be due to "it helps the learning algorithm converge faster, especially for algorithms that are sensitive to feature scales", and that "standard scaling is useful when the features have different scales or when some features have large variances compared to others". So I´m going to check the statistics.


In [ ]:
customer.describe()

### Model Normalised with StandardScaler - Only Numerical

In [ ]:
y = customer['Total Claim Amount']
X = cus_std

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.001)

lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

predictions = lm.predict(X_train)
r2_score(y_train, predictions)

It also gives a score of 1 considering only numerical columns.

### Dropping outliers - Normalised with MinMax

In [31]:
customer = customer[(customer['Total Claim Amount']>lower_limit)&(cus_num['Total Claim Amount']<2300)]

C:\Users\PC\AppData\Local\Temp\ipykernel_11860\3284333650.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  customer = customer[(customer['Total Claim Amount']>lower_limit)&(cus_num['Total Claim Amount']<2300)]


In [33]:
cus_cat = customer.select_dtypes(include = np.object)
cus_one = cus_cat.drop(['Customer', 'Effective To Date'], axis=1)
encoder = OneHotEncoder(drop='first').fit(cus_one)
encoded = encoder.transform(cus_one).toarray()
onehot_encoded = pd.DataFrame(encoded, columns=encoder.get_feature_names_out())

cus_num = customer.select_dtypes(include = np.number)
y = cus_num['Total Claim Amount']
X = cus_num.drop(['Total Claim Amount'], axis=1)
transformer = MinMaxScaler().fit(cus_num)
x_normalized = transformer.transform(cus_num)
cus_nor = pd.DataFrame(x_normalized, columns=cus_num.columns)

customer2 = pd.concat([cus_nor, onehot_encoded], axis=1)
customer2.head()

C:\Users\PC\AppData\Local\Temp\ipykernel_11860\190588800.py:1: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cus_cat = customer.select_dtypes(include = np.object)


,Customer Lifetime Value,Income,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Total Claim Amount,State_California,State_Nevada,...,Sales Channel_Branch,Sales Channel_Call Center,Sales Channel_Web,Vehicle Class_Luxury Car,Vehicle Class_Luxury SUV,Vehicle Class_SUV,Vehicle Class_Sports Car,Vehicle Class_Two-Door Car,Vehicle Size_Medsize,Vehicle Size_Small
0,0.010629,0.562847,0.033755,0.914286,0.050505,0.0,0.000,0.167665,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
1,0.062406,0.000000,0.139241,0.371429,0.424242,0.0,0.875,0.493070,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.134960,0.487763,0.198312,0.514286,0.383838,0.0,0.125,0.246836,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
3,0.070589,0.000000,0.189873,0.514286,0.656566,0.0,0.750,0.230889,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
4,0.011245,0.438443,0.050633,0.342857,0.444444,0.0,0.000,0.060157,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [34]:
y = customer['Total Claim Amount']
X = customer2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)
predictions = lm.predict(X_train)
r2_score(y_train, predictions)

1.0

In [ ]:
predictions_test = lm.predict(X_test)
r2_score(y_test, predictions_test)